In [ ]:
# 2-dimensional data set
# knn: train different metrics
# knn: examine results of changes in metrics

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [3]:
# m a ke  b l o b s
def create_data_set():
    # 2-dimensional, 3 classes, uneven, overlapping
    # n features = n dimensions
    class_1 = sklearn.datasets.make_blobs(n_samples = 1000, centers = 1, n_features = 2, clusters_std = 0.75, random_state = 0)
    class_2 = sklearn.datasets.make_blobs(n_samples = 1000, centers = 1, n_features = 2, clusters_std = 0.75, random_state = 0)
    class_3 = sklearn.datasets.make_blobs(n_samples = 1000, centers = 1, n_features = 2, clusters_std = 0.75, random_state = 0)
